In [5]:
from importlib import reload
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re

In [9]:
# from binance.client import Client
# from binance import enums
import binance
api_key = "your key"
api_secret = "your secret"
client = binance.client(api_key, api_secret)

AttributeError: module 'binance' has no attribute 'client'

In [ ]:
import cryptoz
from cryptoz import stats, utils, plot

# params

In [ ]:
pairs = [
    'BNB/USDT',
    'BTC/USDT',
    'XRP/USDT', 
    'ETH/USDT', 
    'ADA/USDT', 
    'XLM/USDT', 
    'XVG/USDT', 
    'LTC/USDT', 
    'IOTA/USDT', 
    'QTUM/USDT', 
    'NEO/USDT', 
    'DASH/USDT', 
    'ETC/USDT', 
    'LSK/USDT', 
    'XMR/USDT', 
    'OMG/USDT', 
    'STRAT/USDT', 
    'ZEC/USDT',
    'BCC/USDT'
]

In [ ]:
interval = enums.KLINE_INTERVAL_1HOUR
last_window = pd.Timedelta(days=3)

# data

In [ ]:
from cryptoz.exchanges.binance import Binance

binance = Binance(client)

ohlc = binance.get_ohlc(pairs, interval=interval)

In [ ]:
orderbooks = binance.get_orderbooks(pairs)

# basics

## klines

In [ ]:
window = 20
reload(plot)

df = utils.to_df(ohlc, 'C')
sma = utils.cut_df(df.rolling(window).mean(), last_window)
sms = utils.cut_df(df.rolling(window).std(), last_window)
bands = (sma + 2 * sms, sma - 2 * sms)
df = utils.cut_df(df, last_window)

rank = lambda sr: -utils._normalize(sr, 'minmax')[-1]
plot.timesr_matrix(df, bands=bands, rank=rank)

## price

In [ ]:
window = 20

df = utils.to_df(ohlc, 'C')
# price relative to mean of previous prices
df = utils.rolling_normalize(df, 'mean', window)
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

In [ ]:
df = utils.to_df(ohlc, 'C')
df = df.pct_change().dropna()
df.iloc[-1].sort_values()

In [ ]:
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

## volatility

In [ ]:
df = utils.to_df(ohlc, 'C')
df = df.std() / df.mean()
df.sort_values()

In [ ]:
window = 20

df = utils.to_df(ohlc, 'C')
df = df.rolling(window).std()
df = df.pct_change().dropna()
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

## volume

In [ ]:
window = 20

df = utils.to_df(ohlc, 'V')
# volume relative to mean of previous volumes
df = utils.rolling_normalize(df, 'mean', window)
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

# drawdown

## global

In [ ]:
stats.dd.now(ohlc)

## local

In [ ]:
recent_ohlc = utils.cut(ohlc, last_window)
df = stats.dd.from_ohlc(recent_ohlc)

plot.evolution(df, plt.cm.gist_earth_r, rank='last')

# correlation

In [ ]:
window = 20
base_pair = 'BTC/USDT'

df = utils.to_df(ohlc, 'C').pct_change().dropna()
df = stats.corr.rolling(df, window=window)
regex = re.compile(r'(%s-.+)' % base_pair)
df = df[list(filter(regex.search, df.columns))]
df.columns = list(map(lambda x: re.sub(r'(%s)' % base_pair.split('/')[1], '', x), df.columns))
df.dropna(inplace=True)
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.gist_earth, rank='last')

# depth

In [ ]:
plot.depth(orderbooks, ['green', 'orangered'], rank=lambda sr: 1 - sr.max() / abs(sr.min()))

# anomaly detection

## price

In [ ]:
window = 20
levels = [1, 2, 3, 4, 5]

df = utils.to_df(ohlc, 'C')
df = utils.rolling_normalize(df, 'std', window)
df.dropna(inplace=True)
df = utils.classify(df, levels, bipolar=True)
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

## volume

In [ ]:
window = 20
levels = [1, 2, 3, 4, 5]

df = utils.to_df(ohlc, 'V')
df = utils.rolling_normalize(df, 'std', window)
df.dropna(inplace=True)
df = utils.classify(df, levels, bipolar=True)
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

# oscillators

## *RSI*

In [ ]:
def RSI(df, window):
    delta = df.diff().dropna()
    up, down = delta * 0, delta * 0
    pos_mask = delta > 0
    neg_mask = delta < 0
    up[pos_mask] = delta[pos_mask]
    down[neg_mask] = -delta[neg_mask]
    up.iloc[window - 1] = up.iloc[:window].mean()
    up = up.iloc[window - 1:]
    down.iloc[window - 1] = down.iloc[:window].mean()
    down = down.iloc[window - 1:]
    rs = up.ewm(com=window - 1, adjust=False).mean() / down.ewm(com=window - 1, adjust=False).mean()
    rsi = 100 - 100 / (1 + rs)
    return rsi.reindex(df.index)

In [ ]:
window = 20
oversold = 30
overbought = 70

df = utils.to_df(ohlc, 'C')
df = RSI(df, window)
df = utils.trunk(df, [oversold, overbought])
df = utils.rescale(df, [-1, 1])
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

## *CCI*

In [ ]:
def cci(df, period):
    # (Typical Price  -  20-period SMA of TP) / (.015 x Mean Deviation)
    tp_sma = df.rolling(period).mean()
    md = df.rolling(period).apply(lambda a: np.fabs(a - a.mean()).mean())
    return (df - tp_sma) / (.015 * md)

In [ ]:
window = 20
oversold = -300
overbought = 300

df = utils.to_df(ohlc, 'C')
df = cci(df, window)
df = utils.trunk(df, [oversold, overbought])
df = utils.rescale(df, [-1, 1])
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

# signals

## *EMA crossover*

In [ ]:
def ema_crossover(df, ema_fast, ema_slow):
    fast = df.ewm(span=ema_fast, adjust=False).mean()
    slow = df.ewm(span=ema_slow, adjust=False).mean()
    df = fast - slow
    df[df > 0] = 1
    df[df < 0] = -1
    df = df.fillna(0)
    df = df.diff().dropna()
    pos_mask = df == 2
    neg_mask = df == -2
    df[pos_mask] = 1
    df[neg_mask] = -1
    df[~neg_mask & ~pos_mask] = 0
    return df

In [ ]:
fast_window = 7
slow_window = 25

df = utils.to_df(ohlc, 'M')
df = ema_crossover(df, fast_window, slow_window)
df = utils.cut_df(df, last_window)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

## *BollingerBand*

### How wide is the BB?

In [ ]:
def BB(df, window, k):
    sma = df.rolling(window=window).mean()
    std = df.rolling(window=window).std()
    BOLU = sma + std * k
    BOLD = sma - std * k
    return BOLD, BOLU

def corridor(BOLD, BOLU):
    return BOLU - BOLD

In [ ]:
window = 20
k = 2

df = utils.to_df(ohlc, 'C')
df = corridor(*BB(df, window, k)).dropna()
df = utils.rolling_normalize(df, 'mean', window).dropna()
df1 = utils.cut_df(df, last_window)

plot.evolution(df1, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

### Does the price exceed the BB?

In [ ]:
reload(utils)
df = utils.to_df(ohlc, 'C')
df = utils.range_rescale(df, BB(df, window, k), [-1, 1]).dropna()
df2 = utils.cut_df(df, last_window)

plot.evolution(df2, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')

### When to buy/sell?

In [ ]:
df = utils.combine_scales(df1, df2, 'subtract', [-1, 1])
df = utils.classify(df, [0.9], bipolar=True)

plot.evolution(df, plt.cm.Spectral, plot.midpoint_norm(0), rank='last')